In [1]:
import tensorflow as tf
import numpy as np
from tensorflow.contrib import rnn
import itertools as ittls
from collections import Counter

In [6]:
class dynamic_RNN(object):
    def __init__(self,seq_max_len,input_dim,hidden_size,class_num,dynamic=True):
        self.seq_max_len = seq_max_len
        self.input_dim = input_dim
        self.hidden_size = hidden_size
        self.class_num = class_num
        
        ## input place holder for the sequences and length for each tensor
        self.seqlen = tf.placeholder(tf.int32, [None])
        self.input_x = tf.placeholder(dtype = tf.float32, shape = [None, self.seq_max_len, self.input_dim])
        self.label_y = tf.placeholder(dtype = tf.float32, shape = [None, self.class_num])
        
        self.softmax_weight = tf.Variable(tf.random_normal([self.hidden_size,self.class_num]))
        self.softmax_bias = tf.Variable(tf.random_normal([self.class_num]))
        self.lstm_cell = tf.contrib.rnn.BasicLSTMCell(self.hidden_size)

        if dynamic:
            x = tf.unstack(self.input_x, self.seq_max_len, 1)
            outputs, states = tf.nn.dynamic_rnn(self.lstm_cell, self.input_x, dtype=tf.float32,
                                    sequence_length=self.seqlen)
        else:
            x = tf.unstack(self.input_x, self.seq_max_len, 1)
            outputs, states = tf.nn.static_rnn(self.lstm_cell, x, dtype=tf.float32,
                                    sequence_length=self.seqlen)
        outputs = tf.transpose(tf.stack(outputs), [1, 0, 2])
        batch_size = tf.shape(outputs)[0]
        index = tf.range(0, batch_size) * self.seq_max_len + (self.seqlen - 1)
        self.outputs = tf.gather(tf.reshape(outputs, [-1, self.hidden_size]), index)
        
        logits = tf.matmul(self.outputs,self.softmax_weight) + self.softmax_bias
        self.prediction = tf.nn.softmax(logits)
        
        correct_pred = tf.equal(tf.argmax(self.prediction,1),tf.argmax(self.label_y,1))
        self.accuracy = tf.reduce_mean(tf.cast(correct_pred,tf.float32))
        self.init = tf.global_variables_initializer()
        self.inited = False
        self.trained = False
    
    def fit(sess, x, y, batch_size, learning_rate=0.001, iteration = 500):
        if not self.inited:
            sess.run(self.init)
            self.inited = False
            
tf.reset_default_graph()
## Bug test
seq_max_len = 10 # Sequence max length
n_hidden = 64 # hidden layer num of features
n_classes = 2
dynamic = False
with tf.Graph().as_default():
    drnn = dynamic_RNN(seq_max_len= seq_max_len,input_dim=8,hidden_size=n_hidden,class_num=n_classes)


In [ ]:
seq_max_len = 10 # Sequence max length
n_hidden = 64 # hidden layer num of features
n_classes = 2
dynamic = False
with tf.Graph().as_default():
    with tf.Session() as sess:
        seqlen = tf.placeholder(tf.int32, [None])
        input_x = tf.placeholder("float", [None, seq_max_len, 8])
        
        lstm_cell = tf.contrib.rnn.BasicLSTMCell(n_hidden)

        if dynamic:
            outputs, states = tf.nn.dynamic_rnn(lstm_cell, input_x, dtype=tf.float32,
                                    sequence_length=seqlen)
        else:
            x = tf.unstack(input_x, seq_max_len, 1)
            outputs, states = tf.nn.static_rnn(lstm_cell, x, dtype=tf.float32,
                                    sequence_length=seqlen)
        outputs = tf.transpose(tf.stack(outputs), [1, 0, 2])
        batch_size = tf.shape(outputs)[0]
        index = tf.range(0, batch_size) * seq_max_len + (seqlen - 1)
        outputs = tf.gather(tf.reshape(outputs, [-1, n_hidden]), index)
        
        

In [ ]:
X = np.random.randn(2, 10, 8)
X.reshape(-1,8).shape

In [ ]:
seq = [10,9,8,7]
with tf.Graph().as_default():
    with tf.Session() as sess:
        out_put = tf.constant(np.random.randn(4, 10, 8),dtype = tf.float32)
        seqlen = tf.placeholder(tf.int32,[None])
        print seqlen
        fed_dict= {seqlen:seq}
        index = tf.range(0,4) * 10 + (seqlen - 1)
        print sess.run(tf.range(0,4) * 10 ,feed_dict=fed_dict)
        print sess.run(seqlen-1,feed_dict=fed_dict)
        print sess.run(index ,feed_dict = fed_dict)
        reshape = tf.reshape(out_put,[-1,8])
        print sess.run(tf.shape(reshape),feed_dict=fed_dict)
        print sess.run(tf.gather(reshape,index),feed_dict=fed_dict)